In [ ]:
import pandas as pd
%pip install IMDbPY
from imdb import IMDb

Note: you may need to restart the kernel to use updated packages.


In [3]:
movie_df_for_db = pd.read_csv(r'D:\datasets\archive\movies.csv')
movie_df_for_db

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [5]:
rating_full_df_for_db = pd.read_csv(r'D:\datasets\archive\ratings.csv')
rating_df = rating_full_df_for_db[:1000000]
rating_df.drop(['timestamp', 'userId'], axis=1, inplace =True)
rating_df

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3432\2070447989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df.drop(['timestamp', 'userId'], axis=1, inplace =True)


,movieId,rating
0,296,5.0
1,306,3.5
2,307,5.0
3,665,5.0
4,899,3.5
...,...,...
999995,3566,2.0
999996,3578,5.0
999997,3593,2.0
999998,3615,4.0


In [6]:
df_movie_for_db = movie_df_for_db.merge(rating_df, on='movieId', how='left')
df_movie_for_db.dropna(inplace=True)

In [8]:
average_ratings = df_movie_for_db.groupby(['movieId', 'title'])['rating'].mean().reset_index()
df = average_ratings.merge(movie_df_for_db, on='movieId', how='left')
df.drop('title_y', axis=1, inplace=True)
df

,movieId,title_x,rating,genres
0,1,Toy Story (1995),3.893554,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),3.310185,Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),3.162768,Comedy|Romance
3,4,Waiting to Exhale (1995),2.936170,Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),3.135542,Comedy
...,...,...,...,...
21947,208002,The Kill Team (2019),3.500000,Drama|War
21948,208080,Bruja (2019),3.000000,Drama|Fantasy|Thriller
21949,208737,Midway (2019),3.500000,Action|Drama|War
21950,208793,Watchman (2019),3.500000,Drama|Thriller


In [9]:
def get_large_pict_movie(name):
    ia = IMDb()
    try:
        movie = ia.search_movie(name)
        if not movie:
            return np.nan  
        ia.update(movie[0], info=['main'])
        cover_url = movie[0].get('full-size cover url')
        
        if cover_url:
            return cover_url  
        
        return movie[0].get('cover url', np.nan)
    
    except Exception as e:
        print(f"Error: {e}")
        return np.nan

In [ ]:
df['picture_url'] = df['title_x'].apply(get_large_pict_movie)

In [11]:
unique_genres = set()
df['genres'] =df['genres'].apply(lambda x: x.split('|'))
df['genres'].apply(unique_genres.update)
unique_genres = sorted(unique_genres)

In [12]:
unique_genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']